# EVPD-LIH

In [ ]:
%matplotlib widget
from bmcs_matmod.api import GSM1D_EVPD
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
sp.init_printing()

In [ ]:
gsm = GSM1D_EVPD()
gsm_engine = gsm.F_engine

In [ ]:
gsm_engine.f_expr

In [ ]:
(gamma_mech, L_, dL_dS_, dL_dS_A_, dR_dA_n1), (eps_n, delta_eps, Eps_n, delta_A, delta_t, _, _), Sig_n1, f_n1, R_n1, dR_dA_OI_n1 = gsm_engine.Sig_f_R_dR_n1
gamma_mech

In [ ]:
gsm_engine.F_expr

In [ ]:
L_

In [ ]:
stacked_matrix = sp.Matrix.vstack(gsm_engine.u_vars, gsm_engine.Eps.as_explicit())
stacked_matrix
dot_stacked_matrix = sp.Matrix.vstack(sp.Matrix([[gsm_engine.dot_eps]]), gsm_engine.dot_Eps.as_explicit())
dot_stacked_matrix


$$
E ( \dot{\varepsilon} - \dot{\varepsilon}_\mathrm{p}) - K \dot{z}
$$

In [ ]:
(gsm_engine.f_expr.subs(gsm_engine.subs_Sig_Eps).diff(stacked_matrix).T * dot_stacked_matrix)[0,0]

In [ ]:
dL_dS_

In [ ]:
R_n1_subs = R_n1.replace(sp.sign, lambda x: 1)
R_n1_subs

In [ ]:
delta_A

In [ ]:
sp.simplify(f_n1)


In [ ]:
dR_dA_n1_subs = dR_dA_n1.replace(sp.sign, lambda x: 1)
dR_dA_n1_subs

In [ ]:
Sig_n1

## Monotonic strain-driven loading

In [ ]:
mparams = dict(E = 20, K = 5, S = 1, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=10)
n_t = 151
n_I = 1
eps_max = 5.0
final_time = 10
t_t = np.linspace(0, final_time, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
response = gsm.get_response(eps_n_t, t_t, **mparams)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response
# Plot viscoplastic strain and damage along time
eps_t = eps_ta[:, 0]
eps_p_t, omega_t, z_t = Eps_t[:, 0, :].T
sig_p_t, Y_t, Z_t = Sig_t[:, 0, :].T

In [ ]:
fig, ((ax_sig, ax_omega), (ax_lam, ax_z)) = plt.subplots(2, 2, figsize=(12, 8))

# First row: Stress-strain and damage plots
ax_iter = ax_sig.twinx()
ax_eps = ax_omega.twinx()

ax_sig.plot(eps_t, sig_p_t, color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_p_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray', lw=0.5)
ax_iter.set_ylabel(r'$k$', color='gray')

ax_omega.plot(t_t, omega_t, label='Damage', color='brown')
ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.legend()
ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$\omega$', color='brown')
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

# Second row: Lagrange multiplier and plastic strain plots
ax_lam_eps_t = ax_lam.twinx()
ax_lam.plot(t_t, lam_t[:, 0, 0], 'o-', label=r'$\lambda$', color='purple')
ax_lam.set_xlabel(r'$t$')
ax_lam.set_ylabel(r'$\lambda$', color='purple')
ax_lam_eps_t.plot(t_t, eps_t, label=r'$\varepsilon$', color='green', ls='dashed')
ax_lam_eps_t.set_ylabel(r'$\varepsilon$', color='green')

ax_z_eps_p = ax_z.twinx()
ax_z.plot(t_t, z_t, label=r'$z$', color='orange')
ax_z.set_xlabel(r'$t$')
ax_z.set_ylabel(r'$Z$', color='orange')
ax_z_eps_p.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{p}$', color='blue', ls='dashed')
ax_z_eps_p.set_ylabel(r'$\varepsilon_\mathrm{p}$', color='blue')

plt.tight_layout()
plt.show()

## Cyclic strain-driven loading

In [ ]:
t_, sig_max_, t_1_, t_2_, t_3_ = sp.symbols('t sig_max t_1 t_2 t_3', real=True, positive=True)
eps_t_ = sp.Piecewise(
    (sig_max_ / t_1_ * t_, t_ < t_1_),
    (sig_max_, t_ < t_2_),
    (sig_max_ - sig_max_/(t_3_-t_2_)*(t_ - t_2_), t_ < t_3_),
    (0, True)
)
get_eps_t = sp.lambdify((t_, sig_max_, t_1_, t_2_, t_3_), eps_t_, 'numpy')
final_time = 10
eps_max = 3
n_t = 151
#n_t = 31
t_t = np.linspace(0, final_time, n_t)
eps_n_t = get_eps_t(t_t, eps_max, 0.25*final_time, 0.5*final_time, 0.75*final_time)
fig, ax = plt.subplots(1,1)
ax.plot(t_t, eps_n_t)

In [ ]:
mparams = dict(E = 20, K = 5, S = 10, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=10)
response = gsm.get_response(eps_n_t, t_t, **mparams)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response
# Plot viscoplastic strain and damage along time
eps_t = eps_ta[:, 0]
eps_p_t, omega_t, z_t = Eps_t[:, 0, :].T
sig_p_t, Y_t, Z_t = Sig_t[:, 0, :].T

In [ ]:
fig, ((ax_sig, ax_omega), (ax_lam, ax_z)) = plt.subplots(2, 2, figsize=(12, 8))

# First row: Stress-strain and damage plots
ax_iter = ax_sig.twinx()
ax_eps = ax_omega.twinx()

ax_sig.plot(eps_t, sig_p_t, color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_p_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray', lw=0.5)
ax_iter.set_ylabel(r'$k$', color='gray')

ax_omega.plot(t_t, omega_t, label='Damage', color='brown')
ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.legend()
ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$\omega$', color='brown')
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

# Second row: Lagrange multiplier and plastic strain plots
ax_lam_eps_t = ax_lam.twinx()
ax_lam.plot(t_t, lam_t[:, 0, 0], 'o-', label=r'$\lambda$', color='purple')
ax_lam.set_xlabel(r'$t$')
ax_lam.set_ylabel(r'$\lambda$', color='purple')
ax_lam_eps_t.plot(t_t, eps_t, label=r'$\varepsilon$', color='green', ls='dashed')
ax_lam_eps_t.set_ylabel(r'$\varepsilon$', color='green')

ax_z_eps_p = ax_z.twinx()
ax_z.plot(t_t, Z_t, label=r'$z$', color='orange')
ax_z.set_xlabel(r'$t$')
ax_z.set_ylabel(r'$Z$', color='orange')
ax_z_eps_p.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{p}$', color='blue', ls='dashed')
ax_z_eps_p.set_ylabel(r'$\varepsilon_\mathrm{p}$', color='blue')

plt.tight_layout()
plt.show()

Damage is allowed to grow only if
$$
\| \dot{\varepsilon}_\mathrm{e} \| > 0
$$
Can this condition be expressed using the yield function? Why is $\dot{f} < 0 $ not covering the considered case? Because of the dynamic effect?
$$
\| \dot{\varepsilon}_\mathrm{e} \| = 
\| \dot{\varepsilon} -  
\dot{\varepsilon}_\mathrm{p} \| 
$$
If $\| \dot{\varepsilon} \| < 0$ the damage must not evolve. 